# Getting Financial Data - Pandas Datareader

### Introduction:

This time you will get data from a website.


### Step 1. Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import datetime

### Step 2. Create your time range (start and end variables). The start date should be 01/01/2015 and the end should today (whatever your today is).

In [2]:
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime.today()

### Step 3. Get an API key for one of the APIs that are supported by Pandas Datareader, preferably for AlphaVantage.

If you do not have an API key for any of the supported APIs, it is easiest to get one for [AlphaVantage](https://www.alphavantage.co/support/#api-key). (Note that the API key is shown directly after the signup. You do *not* receive it via e-mail.)

(For a full list of the APIs that are supported by Pandas Datareader, [see here](https://pydata.github.io/pandas-datareader/readers/index.html). As the APIs are provided by third parties, this list may change.)

### Step 4. Use Pandas Datarader to read the daily time series for the Apple stock (ticker symbol AAPL) between 01/01/2015 and today, assign it to df_apple and print it.

In [4]:
api_key = '9K57KVUCX5GUISCH'
df_apple = web.DataReader('AAPL', 'av-daily', start, end, api_key=api_key)
print(df_apple.head())

              open    high      low   close    volume
2015-01-02  111.39  111.44  107.350  109.33  53204626
2015-01-05  108.29  108.65  105.410  106.25  64285491
2015-01-06  106.54  107.43  104.630  106.26  65797116
2015-01-07  107.20  108.20  106.695  107.75  40105934
2015-01-08  109.23  112.15  108.700  111.89  59364547


### Step 5. Add a new column "stock" to the dataframe and add the ticker symbol

In [5]:
df_apple['stock'] = 'AAPL'
print(df_apple.columns)

Index(['open', 'high', 'low', 'close', 'volume', 'stock'], dtype='object')


### Step 6. Repeat the two previous steps for a few other stocks, always creating a new dataframe: Tesla, IBM and Microsoft. (Ticker symbols TSLA, IBM and MSFT.)

In [6]:
df_tesla = web.DataReader('TSLA', 'av-daily', start, end, api_key=api_key)
df_tesla['stock'] = 'TSLA'

df_ibm = web.DataReader('IBM', 'av-daily', start, end, api_key=api_key)
df_ibm['stock'] = 'IBM'

df_microsoft = web.DataReader('MSFT', 'av-daily', start, end, api_key=api_key)
df_microsoft['stock'] = 'MSFT'

print(df_tesla.columns)
print(df_ibm.columns)
print(df_microsoft.columns)


Index(['open', 'high', 'low', 'close', 'volume', 'stock'], dtype='object')
Index(['open', 'high', 'low', 'close', 'volume', 'stock'], dtype='object')
Index(['open', 'high', 'low', 'close', 'volume', 'stock'], dtype='object')


### Step 7. Combine the four separate dataFrames into one combined dataFrame df that holds the information for all four stocks

In [7]:
df = pd.concat([df_apple, df_tesla, df_ibm, df_microsoft])

print(df.columns)
df = df.reset_index()

if 'date' not in df.columns:
    df = df.rename(columns={'index': 'date'})

Index(['open', 'high', 'low', 'close', 'volume', 'stock'], dtype='object')


### Step 8. Shift the stock column into the index (making it a multi-level index consisting of the ticker symbol and the date).

In [8]:
df = df.set_index(['stock', 'date'])
print(df.head())

                    open    high      low   close    volume
stock date                                                 
AAPL  2015-01-02  111.39  111.44  107.350  109.33  53204626
      2015-01-05  108.29  108.65  105.410  106.25  64285491
      2015-01-06  106.54  107.43  104.630  106.26  65797116
      2015-01-07  107.20  108.20  106.695  107.75  40105934
      2015-01-08  109.23  112.15  108.700  111.89  59364547


### Step 7. Create a dataFrame called vol, with the volume values.

In [9]:
vol = df['volume']
print(vol.head())

stock  date      
AAPL   2015-01-02    53204626
       2015-01-05    64285491
       2015-01-06    65797116
       2015-01-07    40105934
       2015-01-08    59364547
Name: volume, dtype: int64


### Step 8. Aggregate the data of volume to weekly.
Hint: Be careful to not sum data from the same week of 2015 and other years.

In [10]:
df.index = df.index.set_levels([df.index.levels[0], pd.to_datetime(df.index.levels[1])])
vol_weekly = df['volume'].groupby([pd.Grouper(level='stock'), pd.Grouper(level='date', freq='W')]).sum()
print(vol_weekly.head())

stock  date      
AAPL   2015-01-04     53204626
       2015-01-11    282868187
       2015-01-18    304226647
       2015-01-25    198737041
       2015-02-01    465842684
Name: volume, dtype: int64


### Step 9. Find all the volume traded in the year of 2015

In [21]:
vol_2015 = df['volume'].loc[(slice(None), slice('2015-01-01', '2015-12-31'))]
result = vol_2015.groupby(level='stock').sum()
print (result)

stock
AAPL    13064316775
IBM      1105545521
MSFT     9057582311
TSLA     1086708380
Name: volume, dtype: int64
